# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers for each contest
    - reported winner(s) for each contest
    - an upper bound on the number of ballot cards that contain each contest
    - an upper bound on the total number of cards across all contests
    - whether to use card style information to target sampling
+ a ballot manifest (see below)
+ a random seed
+ a file of cast vote records (for ballot-comparison audits)
+ reported vote tallies for for each contest (for ballot-polling audits of plurality, supermajority, and approval social choice functions)
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper cards selected for audit

`use_style` controls whether the sample is drawn from all cards (`use_style == False`) or card style information is used
to target the cards that purport to contain each contest (`use_style == True`).
In the current implementation, card style information is inferred from cast-vote records, with additional 'phantom' CVRs if there could be more cards that contain a contest than is accounted for in the CVRs.
Errors in the card style information are treated conservatively using the  "phantoms-to-evil-zombies" (~2EZ) approach ([Banuelos & Stark, 2012](https://arxiv.org/abs/1207.3413)) so that the risk limit remains valid, even if the CVRs misrepresent
which cards contain which contests.

The two ways of sampling are treated differently. 
If the sample is to be drawn only from cards that--according to the CVR--contain a particular contest, and a sampled card turns out not to
contain that contest, that is considered a discrepancy, dealt with using the ~2EZ approach.
It is assumed that every CVR corresponds to a card in the manifest, but there might
be cards cast in the contest for which there is no corresponding CVR. In that case,
phantom CVRs are created to ensure that the audit is still truly risk-limiting.

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of CVRs that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the ~2EZ approach to deal with missing CVRs. This can greatly increase the efficiency of the audit if 
some contests appear on only a small percentage of the cast cards ([Glazer, Spertus, and Stark, 2021](https://dl.acm.org/doi/10.1145/3457907)).
If there are more CVRs than the upper bound on the number of cards, extra CVRs can be deleted provided
that deletion does not change any contest outcome. See [Stark, 2022](https://arxiv.org/abs/2207.01362).
(However, if there more CVRs than cards, that is evidence of a process failure.)

Any sampled phantom card (i.e., a card for which there is no CVR) is treated as if its CVR is a non-vote (which it is), and as if its MVR was least favorable (an "evil zombie" producing the greatest doubt in every assertion, separately). Any sampled card for which there is a CVR is compared to its corresponding CVR. 
If the card turns out not to contain the contest (despite the fact that the CVR says it does), the MVR is treated in the least favorable way for each assertion (i.e., as a zombie rather than as a non-vote).

The tool helps select cards for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited cards.

The pre-10/2021 version used a single sample to audit all contests. 

### Internal workflow

+ Read overall audit information (including the seed) and contest information
+ Read assertions for IRV contests and construct assertions for all other contests
+ Read ballot manifest
+ Read cvrs. Every CVR should have a corresponding manifest entry.
+ Prepare ~2EZ:
    - `N_phantoms = max_cards - cards_in_manifest`
    - If `N_phantoms < 0`, complain
    - Else create `N_phantoms` phantom cards
    - For each contest `c`:
        + `N_c` is the input upper bound on the number of cards that contain `c`
        + if `N_c is None`, `N_c = max_cards - non_c_cvrs`, where `non_c_cvrs` is #CVRs that don't contain `c`
        + `C_c` is the number of CVRs that contain the contest
        + if `C_c > N_c`, complain
        + else if `N_c - C_c > N_phantoms`, complain
        + else:
            - Consider contest `c` to be on the first `N_c - C_c` phantom CVRs
            - Consider contest `c` to be on the first `N_c - C_c` phantom ballots
+ Create Assertions for every Contest. This involves also creating an Assorter for every Assertion, and a `NonnegMean` test
for every Assertion.
+ Calculate assorter margins for all assorters:
    - If `not use_style`, apply the Assorter to all cards and CVRs, including phantoms
    - Else apply the assorter only to cards/cvrs reported to contain the contest, including phantoms that contain the contest
+ Set `assertion.test.u` to the appropriate value for each assertion: `assorter.upper_bound` for polling audits or 
      `2/(2-assorter.margin/assorter.upper_bound)` for ballot-level comparison audits
+ Estimate starting sample size for the specified sampling design (w/ or w/o replacement, stratified, etc.), for chosen risk function, use of card-style information, etc.:
    - User-specified criterion, controlled by parameters. Examples:
        + expected sample size for completion, on the assumption that there are no errors
        + 90th percentile of sample size for completion, on the assumption that errors are not more frequent than specified
    - If `not use_style`, base estimate on sampling from the entire manifest, i.e., smallest assorter margin
    - Else use consistent sampling:
        + Augment each CVR (including phantoms) with a probability of selection, `p`, initially 0
        + For each contest `c`:
            - Find sample size `n_c` that meets the criterion 
            - For each non-phantom CVR that contains the contest, set `p = max(p, n_c/N_c)` 
        + Estimated sample size is the sum of `p` over all non-phantom CVRs
+ Draw the random sample:
    - Use the specified design, including using consistent sampling for style information
    - Express sample cards in terms of the manifest
    - Export
+ Read manual interpretations of the cards (MVRs)
+ Calculate attained risk for each assorter
    - Use ~2EZ to deal with phantom CVRs or cards; the treatment depends on whether `use_style == True`
+ Report
+ Estimate incremental sample size if any assorter nulls have not been rejected
+ Draw incremental sample; etc

# Audit parameters.

The overall audit involves information that is the same across contests, encapsulated in
a dict called `audit`:

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample (for SHA256 PRNG)
* `sim_seed`: seed for simulations to estimate sample sizes (for Mersenne Twister PRNG)
* `quantile`: quantile of the sample size to use for setting initial sample size
* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `use_style`: Boolean. If True, use card style information (inferred from CVRs) to target samples. If False, sample from all cards, regardless of the contest.
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)
* `error_rate_1`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `error_rate_2`: expected rate of 2-vote overstatements. 2-vote overstatements should be extremely rare.
Recommended value: 0. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `reps`: number of replications to use to estimate sample sizes. If `reps is None`, uses a deterministic method
* `quantile`: quantile of sample size to estimate. Not used if `reps is None`
* `strata`: a dict describing the strata. Keys are stratum identifiers; values are dicts containing:
    + `max_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.
    + `replacement`: whether to sample from this stratum with replacement. 
    + `use_style`: True if the sample in that stratum uses card-style information.
    + `audit_type` one of Contest.POLLING, Contest.CARD_COMPARISON, Contest.BATCH_COMPARISON but only POLLING and CARD_COMPARISON are currently implemented. 
    + `test`: the name of the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_mart`, `alpha_mart`, `betting_mart`. 
Not all risk functions work with every social choice function or every sampling method. 
    + `estim`: the estimator to be used by the `alpha_mart` risk function. Options:  
        - `fixed_alternative_mean` (default)
        - `shrink_trunc`
        - `optimal_comparison`
    + `bet`: the method to select the bet for the `betting_mart` risk function. Options:
        - `fixed_bet` (default)
        - `agrapa`
    + `test_kwargs`: keyword arguments for the risk function

----

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are Contest objects with attributes:
        - `risk_limit`: the risk limit for the audit of this contest
        - `cards`: an upper bound on the number of cast cards that contain the contest
        - `choice_function`: `Audit.SOCIAL_CHOICE_FUNCTION.PLURALITY`, 
          `Audit.SOCIAL_CHOICE_FUNCTION.SUPERMAJORITY`, or `Audit.SOCIAL_CHOICE_FUNCTION.IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV, aka STV, is not supported)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
           (share_to_win*n_winners must be less than 100%)
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won.
           Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions
        - `audit_type`: the audit strategy. Currently `Audit.AUDIT_TYPE.POLLING (ballot-polling)`, 
           `Audit.AUDIT_TYPE.CARD_COMPARISON` (ballot-level comparison audits), and `Audit.AUDIT_TYPE.ONEAUDIT`
            are implemented. HYBRID and STRATIFIED are planned.
        - `test`: the risk function for the audit. Default is `NonnegMean.alpha_mart`, the alpha supermartingale test
        - `estim`: estimator for the alternative hypothesis for the test. Default is `NonnegMean.shrink_trunc`
        - `use_style`: True to use style information from CVRs to target the sample. False for polling audits or for sampling from all ballots for every contest.
        - other keys and values are added by the software, including `cvrs`, the number of CVRs that contain the contest, and `p`, the sampling fraction expected to be required to confirm the contest

In [36]:
# if shangrla has not already been installed, install it then restart the kernel
# !pip install -e "../"

In [37]:
import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256, int_from_hash
from cryptorandom.sample import sample_by_index

from shangrla.core.Audit import Audit, Assertion, Assorter, Contest, CVR, Stratum
from shangrla.core.NonnegMean import NonnegMean
from shangrla.core.Dominion import Dominion
from shangrla.core.Hart import Hart

In [38]:
audit = Audit.from_dict({
         'seed':           12345678901234567890,
         'sim_seed':       314159265,
         'cvr_file':       './Data/SFDA2019/SFDA2019_PrelimReport12VBMJustDASheets.raire',
         'manifest_file':  './Data/SFDA2019/N19 ballot manifest with WH location for RLA Upload VBM 11-14.xlsx',
         'sample_file':    './Data/sample.csv',
         'mvr_file':       './Data/mvr.json',
         'log_file':       './Data/log.json',
         'quantile':       0.8,
         'error_rate_1':   0.001,
         'error_rate_2':   0.0,
         'reps':           100,
         'strata':         {'stratum_1': {'max_cards':   293555, 
                                          'use_style':   True,
                                          'replacement': False,
                                          'audit_type':  Audit.AUDIT_TYPE.CARD_COMPARISON,
                                          'test':        NonnegMean.alpha_mart,
                                          'estimator':   NonnegMean.optimal_comparison,
                                          'test_kwargs': {}
                                         }
                           }
        })

# find upper bound on total cards across strata
audit.max_cards = np.sum([s.max_cards for s in audit.strata.values()])
'''
max_cards = 293555 # 146662 VBM turnout per SF Elections release 12 
https://sfelections.sfgov.org/november-5-2019-election-results-summary
'''

'\nmax_cards = 293555 # 146662 VBM turnout per SF Elections release 12 \nhttps://sfelections.sfgov.org/november-5-2019-election-results-summary\n'

In [39]:
# contests to audit. Edit with details of your contest (eg., Contest 339 is the DA race)
contest_dict = {'339':{
                   'name': 'DA',
                   'risk_limit':       0.05,
                   'cards':            146662,
                   'choice_function':  Contest.SOCIAL_CHOICE_FUNCTION.IRV,
                   'n_winners':        1,
                   'candidates':       ['15','16','17','18','45'],
                   'winner':           ['15'],
                   'assertion_file':   './Data/SFDA2019/SF2019Nov8Assertions.json',
                   'audit_type':       Audit.AUDIT_TYPE.CARD_COMPARISON,
                   'test':             NonnegMean.alpha_mart,
                   'estim':            NonnegMean.optimal_comparison
                  }
               }

contests = Contest.from_dict_of_dicts(contest_dict)

Example of other social choice functions:

        contests =  {'city_council':{'name': 'City Council',
                             'risk_limit':0.05,
                             'cards': None,
                             'choice_function': Contest.SOCIAL_CHOICE_FUNCTION.PLURALITY,
                             'n_winners':3,
                             'candidates':['Doug','Emily','Frank','Gail','Harry'],
                             'winner' : ['Doug', 'Emily', 'Frank']
                            },
                        'measure_1':{'name': 'Measure 1',
                             'risk_limit':0.05,
                             'cards': 65432,
                             'choice_function': Contest.SOCIAL_CHOICE_FUNCTION.SUPERMAJORITY,
                             'share_to_win':2/3,
                             'n_winners':1,
                             'candidates':['yes','no'],
                             'winner' : ['yes']
                            }                  
                      }

In [40]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c].choice_function == Contest.SOCIAL_CHOICE_FUNCTION.IRV:
        with open(contests[c].assertion_file, 'r') as f:
            contests[c].assertion_json = json.load(f)['audits'][0]['assertions']

In [41]:
# construct the dict of dicts of assertions for each contest
Assertion.make_all_assertions(contests)

True

In [42]:
audit.check_audit_parameters(contests)

## Read the ballot manifest

In [43]:
# special for Primary/Dominion manifest format
manifest = pd.read_excel(audit.manifest_file)

## Read the CVR data and create CVR objects

In [44]:
# for ballot-level comparison audits
cvr_list, cvrs_read, unique_ids = CVR.from_raire_file(audit.cvr_file)
cvr_list = Dominion.raire_to_dominion(cvr_list)
print(f'Read {cvrs_read} cvrs; {unique_ids} unique CVR identifiers after merging')

Read 146663 cvrs; 146662 unique CVR identifiers after merging


In [45]:
# double-check whether the manifest accounts for every card
audit.max_cards, np.sum(manifest['Total Ballots'])

(293555, 293555)

In [46]:
# Check that there is a card in the manifest for every card (possibly) cast. If not, add phantoms.
manifest, manifest_cards, phantom_cards = Dominion.prep_manifest(manifest, audit.max_cards, len(cvr_list))
manifest

,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_cards
0,1,99808,78,116,3,116
1,1,99808,77,115,3,231
2,1,99808,79,120,3,351
3,1,99808,81,76,3,427
4,1,99808,80,116,3,543
...,...,...,...,...,...,...
5476,3506,99815,86,2,19,292557
5477,3506,99815,84,222,19,292779
5478,3506,99815,83,346,19,293125
5479,3506,99815,82,332,19,293457


## Create CVRs for phantom cards

In [47]:
# For Comparison Audits Only
#----------------------------

# If the sample draws a phantom card, these CVRs will be used in the comparison.
# phantom MVRs should be treated as zeros by the Assorter for every contest

# setting use_style = False to generate phantoms

cvr_list, phantom_vrs = CVR.make_phantoms(audit=audit, contests=contests, cvr_list=cvr_list, prefix='phantom-1-')
print(f"Created {phantom_vrs} phantom records")

Created 0 phantom records


In [48]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
min_margin = Assertion.set_all_margins_from_cvrs(audit=audit, contests=contests, cvr_list=cvr_list)

print(f'minimum assorter margin {min_margin}')
Contest.print_margins(contests)

minimum assorter margin 0.019902906001554532
margins in contest 339:
	assertion 18 v 17 elim 15 16 45: 0.045792366120740224
	assertion 17 v 16 elim 15 18 45: 0.019902906001554532
	assertion 15 v 18 elim 16 17 45: 0.028923647570604505
	assertion 18 v 16 elim 15 17 45: 0.0830003681935334
	assertion 17 v 16 elim 15 45: 0.058079120699294995
	assertion 15 v 17 elim 16 45: 0.08064120222007065
	assertion 15 v 17 elim 16 18 45: 0.10951712099930444
	assertion 18 v 16 elim 15 45: 0.14875018750596603
	assertion 15 v 16 elim 17 45: 0.13548158350492967
	assertion 15 v 16 elim 17 18 45: 0.1365247985163165
	assertion 15 v 16 elim 18 45: 0.16666893946625572
	assertion 15 v 16 elim 45: 0.15626406294745743
	assertion 15 v 45: 0.2956457705472446


In [49]:
audit.write_audit_parameters(contests=contests)

## Set up for sampling

## Find initial sample size

In [50]:
# find initial sample size 
sample_size = audit.find_sample_size(contests, cvrs=cvr_list)  
print(f'{sample_size=}\n{[(i, c.sample_size) for i, c in contests.items()]}')

sample_size=372
[('339', 372)]


## Draw the first sample

In [51]:
# draw the initial sample using consistent sampling
prng = SHA256(audit.seed)
CVR.assign_sample_nums(cvr_list, prng)

True

In [52]:
sampled_cvr_indices = CVR.consistent_sampling(cvr_list=cvr_list, contests=contests)
n_sampled_phantoms = np.sum(sampled_cvr_indices > manifest_cards)
print(f'The sample includes {n_sampled_phantoms} phantom cards.')

The sample includes 0 phantom cards.


In [53]:
len(cvr_list), manifest_cards, audit.max_cards

(146662, 293555, 293555)

In [54]:
# for comparison audit
cards_to_retrieve, sample_order, cvr_sample, mvr_phantoms_sample = \
    Dominion.sample_from_cvrs(cvr_list, manifest, sampled_cvr_indices)

# for polling audit
# cards_to_retrieve, sample_order, mvr_phantoms_sample = Dominion.sample_from_manifest(manifest, sample)

In [55]:
# write the sample
Dominion.write_cards_sampled(audit.sample_file, cards_to_retrieve, print_phantoms=False)

## Read the audited sample data

In [56]:
# for real data
with open(audit.mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

# for simulated data, no errors
mvr_sample = cvr_sample.copy()

## Find measured risks for all assertions

In [57]:
CVR.prep_comparison_sample(mvr_sample, cvr_sample, sample_order)  # for comparison audit
# CVR.prep_polling_sample(mvr_sample, sample_order)  # for polling audit

###### TEST
# permute part of the sample to introduce errors deliberately
mvr_sample = cvr_sample.copy()
n_errs = 5
errs = mvr_sample[0:n_errs].copy()
np.random.seed(12345678)
np.random.shuffle(errs)
mvr_sample[0:n_errs] = errs

In [58]:
p_max = Assertion.set_p_values(contests=contests, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'maximum assertion p-value {p_max}')
done = audit.summarize_status(contests)

maximum assertion p-value 0.48779308744628547

p-values for assertions in contest 339
	18 v 17 elim 15 16 45: 0.47847909464463045
	17 v 16 elim 15 18 45: 0.48779308744628547
	15 v 18 elim 16 17 45: 0.44631352397209006
	18 v 16 elim 15 17 45: 2.877844184074244e-07
	17 v 16 elim 15 45: 0.0035354717267599813
	15 v 17 elim 16 45: 4.0458461637126434e-07
	15 v 17 elim 16 18 45: 2.3358046805505937e-07
	18 v 16 elim 15 45: 5.790407277111407e-13
	15 v 16 elim 17 45: 9.312863307985585e-12
	15 v 16 elim 17 18 45: 1.8495628594508403e-09
	15 v 16 elim 18 45: 1.1604860427653792e-14
	15 v 16 elim 45: 9.55405920633293e-14
	15 v 45: 1.373326785425354e-26

contest 339 audit INCOMPLETE at risk limit 0.05. Attained risk 0.48779308744628547
assertions remaining to be proved:
	18 v 17 elim 15 16 45	contest_id: 339 winner: 18 loser: 17 assorter: contest_id: 339
upper bound: 1, winner defined: False, loser defined: False, assort defined: Truetally_pool_means: None p-value: 0.47847909464463045 margin: 0.045792

In [59]:
# Log the status of the audit 
audit.write_audit_parameters(contests)

# How many more cards should be audited?

Estimate how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* the assumption that we will continue to see errors at the same rate observed in the sample

In [60]:
# Estimate sample size required to confirm the outcome, if errors continue
# at the same rate as already observed.

new_size = audit.find_sample_size(contests, cvrs=cvr_list, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'{new_size=}\n{[(i, c.sample_size) for i, c in contests.items()]}')


new_size=7098
[('339', 6726)]


In [61]:
# save the first sample
sampled_cvr_indices_old, cards_to_retrieve_old, sample_order_old, cvr_sample_old, mvr_phantoms_sample_old = \
    sampled_cvr_indices, cards_to_retrieve,     sample_order,     cvr_sample,     mvr_phantoms_sample

In [62]:
# draw the sample
sampled_cvr_indices = CVR.consistent_sampling(cvr_list=cvr_list, contests=contests)
n_sampled_phantoms = np.sum(sampled_cvr_indices > manifest_cards)
print(f'The sample includes {n_sampled_phantoms} phantom cards.')

# for comparison audit
cards_to_retrieve, sample_order, cvr_sample, mvr_phantoms_sample = \
    Dominion.sample_from_cvrs(cvr_list, manifest, sampled_cvr_indices)

# for polling audit
# cards_to_retrieve, sample_order, mvr_phantoms_sample = Dominion.sample_from_manifest(manifest, sample)

# write the sample
# could write only the incremental sample using list(set(cards_to_retrieve) - set(cards_to_retrieve_old))
Dominion.write_cards_sampled(audit.sample_file, cards_to_retrieve, print_phantoms=False)

The sample includes 0 phantom cards.


In [63]:
# for real data
with open(audit.mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

# for simulated data, no errors
mvr_sample = cvr_sample.copy()

## Find measured risks for all assertions

In [64]:
CVR.prep_comparison_sample(mvr_sample, cvr_sample, sample_order)  # for comparison audit
# CVR.prep_polling_sample(mvr_sample, sample_order)  # for polling audit

###### TEST
# permute part of the sample to introduce errors deliberately
mvr_sample = cvr_sample.copy()
n_errs = 5
errs = mvr_sample[0:n_errs].copy()
np.random.seed(12345678)
np.random.shuffle(errs)
mvr_sample[0:n_errs] = errs

In [65]:
p_max = Assertion.set_p_values(contests=contests, mvr_sample=mvr_sample, cvr_sample=cvr_sample)
print(f'maximum assertion p-value {p_max}')
done = audit.summarize_status(contests)

maximum assertion p-value 1.2395432394747009e-28

p-values for assertions in contest 339
	18 v 17 elim 15 16 45: 1.4449144036994632e-65
	17 v 16 elim 15 18 45: 1.2395432394747009e-28
	15 v 18 elim 16 17 45: 5.2774518365009846e-42
	18 v 16 elim 15 17 45: 6.205447790363212e-127
	17 v 16 elim 15 45: 2.8080650629218943e-86
	15 v 17 elim 16 45: 2.6490812247796027e-123
	15 v 17 elim 16 18 45: 1.7998086930305547e-166
	18 v 16 elim 15 45: 1.7022899132638306e-231
	15 v 16 elim 17 45: 4.74504262834718e-210
	15 v 16 elim 17 18 45: 2.445447527327654e-209
	15 v 16 elim 18 45: 9.191770721839916e-261
	15 v 16 elim 45: 8.342200191989171e-244
	15 v 45: 0.0

contest 339 AUDIT COMPLETE at risk limit 0.05. Attained risk 1.2395432394747009e-28
